# Constructing network from metanetx

In [1]:
library(tidyverse)
library(readxl)
library(stringr)
library(igraph)
library(tidygraph)
library(DT)
library(ChemmineR)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.0  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.2       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘igraph’

The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union

The following objects are masked from ‘package:purrr’:

    compose, simplify

The following object is masked from ‘package:tidyr’:

    crossing

The following object is masked from ‘package:tibble’:

    as_data_frame

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union


Attaching package: ‘tidygraph’

The following object is masked from ‘package:igraph’:

    groups

The following o

ERROR: Error in library(DT): there is no package called ‘DT’


## import data

In [3]:
reactions <- read_tsv("reac_prop.tsv.gz", comment = "#") %>% 
  select_if(~ any(!is.na(.)))

reactions %>% head()

Parsed with column specification:
cols(
  MNX_ID = col_character(),
  Equation = col_character(),
  Description = col_character(),
  Balance = col_character(),
  EC = col_character(),
  Source = col_character()
)


MNX_ID,Equation,Description,Balance,EC,Source
MNXR01,1 MNXM01@MNXD1 = 1 MNXM1@MNXD1,1 `H(+)` = 1 `H(+)`,true,NA,MNXR01
MNXR94667,1 BIOMASS@MNXD1 = 1 BIOMASS@BOUNDARY,1 `Biomass` = 1 `Biomass`,true,NA,BIOMASS_EXT
MNXR94668,1 MNXM3428@MNXD1 = 1 MNXM3428@MNXD2,1 `10fthf5glu` = 1 `10fthf5glu`,true,NA,bigg:10FTHF5GLUtl
MNXR94669,1 MNXM3429@MNXD1 = 1 MNXM3429@MNXD2,1 `10fthf6glu` = 1 `10fthf6glu`,true,NA,bigg:10FTHF6GLUtl
MNXR94670,1 MNXM5422@MNXD1 = 1 MNXM5422@MNXD2,1 `10fthf7glu` = 1 `10fthf7glu`,true,NA,bigg:10FTHF7GLUtl
MNXR94672,1 MNXM237@MNXD1 = 1 MNXM237@MNXD2,1 `(6S)-10-formyltetrahydrofolate` = 1 `(6S)-10-formyltetrahydrofolate`,true,NA,bigg:10FTHFtl


In [4]:
chem_prop <- read_tsv("chem_prop_used2.tsv.gz") %>% 
  rename("name" = "Description")

chem_prop %>% head()

Parsed with column specification:
cols(
  MNX_ID = col_character(),
  Description = col_character(),
  Formula = col_character(),
  Charge = col_double(),
  Mass = col_double(),
  InChI = col_character(),
  SMILES = col_character(),
  Source = col_character(),
  InChIKey = col_character()
)


MNX_ID,name,Formula,Charge,Mass,InChI,SMILES,Source,InChIKey
MNXM0,molecular entity,NA,NA,NA,NA,NA,chebi:23367,NA
MNXM01,H(+),H,1,1.00794,InChI=1S/p+1,[H+],MNXM01,GPRLSGONYQIRFK-UHFFFAOYSA-N
MNXM1,H(+),H,1,1.00739,InChI=1S/p+1,[H+],chebi:15378,GPRLSGONYQIRFK-UHFFFAOYSA-N
MNXM2,H2O,H2O,0,18.01530,InChI=1S/H2O/h1H2,[H]O[H],chebi:15377,XLYOFNOQVPJJNP-UHFFFAOYSA-N
MNXM3,ATP,C10H12N5O13P3,-4,503.15150,"InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15(3-14-5)10-7(17)6(16)4(26-10)1-25-30(21,22)28-31(23,24)27-29(18,19)20/h2-4,6-7,10,16-17H,1H2,(H,21,22)(H,23,24)(H2,11,12,13)(H2,18,19,20)/p-4/t4-,6-,7-,10-/m1/s1",Nc1ncnc2n(cnc12)[C@@H]1O[C@H](COP([O-])(=O)OP([O-])(=O)OP([O-])([O-])=O)[C@@H](O)[C@H]1O,chebi:30616,ZKHQWZAMYRWXGA-KQYNXXCUSA-J
MNXM4,O2,O2,0,31.99880,InChI=1S/O2/c1-2,O=O,chebi:15379,MYMOFIZGZYHOMD-UHFFFAOYSA-N


In [5]:
valid_chem <- read_tsv("all_chem_lt1400_validity.tsv.gz")

valid_chem %>% head()

Parsed with column specification:
cols(
  MNX_ID = col_character(),
  validity = col_logical()
)


MNX_ID,validity
MNXM01,FALSE
MNXM1,FALSE
MNXM2,TRUE
MNXM3,TRUE
MNXM4,TRUE
MNXM5,TRUE


# construct network

In [6]:
react_tbl <- reactions %>% 
  select(MNX_ID, Equation, Balance) %>% 
  separate(Equation, c("Left", "Right"), sep=" = ") %>% 
  mutate(reversible = str_detect(Balance,"true")) 

react_tbl %>% head()

MNX_ID,Left,Right,Balance,reversible
MNXR01,1 MNXM01@MNXD1,1 MNXM1@MNXD1,true,TRUE
MNXR94667,1 BIOMASS@MNXD1,1 BIOMASS@BOUNDARY,true,TRUE
MNXR94668,1 MNXM3428@MNXD1,1 MNXM3428@MNXD2,true,TRUE
MNXR94669,1 MNXM3429@MNXD1,1 MNXM3429@MNXD2,true,TRUE
MNXR94670,1 MNXM5422@MNXD1,1 MNXM5422@MNXD2,true,TRUE
MNXR94672,1 MNXM237@MNXD1,1 MNXM237@MNXD2,true,TRUE


In [7]:
react_tbl2 <- react_tbl %>% 
  filter(reversible == TRUE) %>% 
  dplyr::select(MNX_ID,Right2=Left,Left2=Right) %>% 
  dplyr::select(MNX_ID,Left=Left2, Right=Right2) %>% 
  bind_rows(react_tbl)

react_tbl2 %>% 
  dplyr::select(-reversible) %>% 
  mutate(Left=strsplit(Left," \\+ ")) %>% 
  mutate(Right=strsplit(Right," \\+ ")) %>% 
  dplyr::select(MNX_ID,Left) %>% 
  unnest(Left) %>% 
  dplyr::select(from=Left, to=MNX_ID) %>% 
  separate(from, into = c("stoch","from"), extra="drop") %>% 
  select(from,to) -> substrates

substrates %>% head()

from,to
MNXM1,MNXR01
BIOMASS,MNXR94667
MNXM3428,MNXR94668
MNXM3429,MNXR94669
MNXM5422,MNXR94670
MNXM237,MNXR94672


In [8]:
react_tbl2 %>% 
  dplyr::select(-reversible) %>% 
  mutate(Left=strsplit(Left," \\+ ")) %>% 
  mutate(Right=strsplit(Right," \\+ ")) %>% 
  dplyr::select(MNX_ID,Right) %>% 
  unnest(Right) %>% 
  dplyr::select(from=MNX_ID, to=Right) %>% 
  separate(to, into = c("stoch","to"), extra="drop") %>% 
  select(from,to)-> products

products %>% head()

from,to
MNXR01,MNXM01
MNXR94667,BIOMASS
MNXR94668,MNXM3428
MNXR94669,MNXM3429
MNXR94670,MNXM5422
MNXR94672,MNXM237


In [9]:
compound_network_fixed <- 
  inner_join(substrates,products,by=c("to"="from")) %>% 
  rename(MNX_ID=to) %>% 
  dplyr::select(from,to=to.y,MNX_ID)  %>% 
  filter(from != to) %>% 
  mutate(from=trimws(from),
         to=trimws(to)) %>% arrange(MNX_ID) %>% 
  as_tbl_graph()

compound_network_fixed

# A tbl_graph: 29293 nodes and 640464 edges
#
# A directed multigraph with 172 components
#
# Node Data: 29,293 x 1 (active)
  name     
  <chr>    
1 MNXM1    
2 MNXM01   
3 MNXM17   
4 MNXM8415 
5 MNXM10958
6 MNXM47   
# … with 2.929e+04 more rows
#
# Edge Data: 640,464 x 3
   from    to MNX_ID    
  <int> <int> <chr>     
1     1     2 MNXR01    
2     2     1 MNXR01    
3     1     5 MNXR100000
# … with 6.405e+05 more rows

# reactant pair similarity

takes time, so only 100 lines are taken into account

In [11]:
compound_network_fixed %>% 
  left_join(chem_prop, by=c("name"="MNX_ID")) %>% 
  left_join(valid_chem, by=c("name"="MNX_ID")) %>% 
  filter(!is.na(SMILES)) %>%    # TODO check if this is okay!
  filter(Mass < 1400) %>%     # discard too large molecules
  filter(validity==TRUE) %>%  # discard molecules which have invalid SDF
  select(name,name.y,SMILES) %>% 
  activate(edges) %>% 
  mutate(fsmiles= .N()$SMILES[from],
         tsmiles= .N()$SMILES[to],
         fname= .N()$name[from],
         tname= .N()$name[to]) %>% 
  as_tibble() %>% 
  head(100) -> test_df3


tanimoto similarity function is defined as:

In [12]:
smiles2tanimoto2 <- function(mol1,mol2){

  sdfset <- smiles2sdf(c(cmp1=mol1,cmp2=mol2))
  validity <- validSDF(sdfset)
  if(!all(validity)) {return(-1.0)}
  
  apset <- sdf2ap(sdfset)
  fpset <- desc2fp(x=apset, descnames=512, type="FPset")
  result <- fpSim(fpset[1], fpset[2], method="Tanimoto")
  result
}

`furrr` will work in parallel locally but probably won't benefit when run at Binder

In [14]:
#library(furrr)
#future::plan(future::multicore) 

test_df3 %>% 
  mutate(tanim_sim = map2_dbl(fsmiles,tsmiles,~ smiles2tanimoto2(.x,.y))) %>% 
  select(-contains("smiles"))

ERROR: Error: Evaluation error: could not find function "smiles2sdf".
